In [1]:
from hail import *
hc = HailContext(tmp_dir = '/nvme/tmp/shusson')

Running on Apache Spark version 2.1.0
SparkUI available at http://129.94.72.55:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.1-c72c76d


In [2]:
vds = hc.read('/nvme/tmp/shusson/sample_stats.vds')

In [3]:
vds.summarize()

Summary(samples=2845, variants=84222, call_rate=0.989135, contigs=['X', '12', '8', '19', '4', '15', '11', '9', 'Y', '22', '13', '16', '5', '10', '21', '6', '1', '17', '14', 'MT', '20', '2', '18', '7', '3'], multiallelics=0, snps=66323, mnps=0, insertions=8225, deletions=9674, complex=0, star=0, max_alleles=2)

In [4]:
if not vds.was_split():
    vds = vds.split_multi()

vds = vds.annotate_variants_expr('va.allele = va.alleles[va.aIndex-1]')

In [5]:
kt=vds.variants_table()

kt = kt.annotate('VARIANT=v, \
CHROMOSOME=v.contig, \
START=v.start, \
REF=v.ref, \
ALT=v.alt, \
RSID=orElse(va.allele.annotation.rsid, "\\\N"), \
AC=va.allele.metrics.allele_counts.total.toInt(), \
AF=if (isMissing(va.allele.metrics.allele_frequencies.total)) "\\\N" else str(va.allele.metrics.allele_frequencies.total),\
nHomRef=va.extra.nHomRef.toInt(), \
nHet=va.extra.nHet.total.toInt(), \
nHomVar=va.extra.nHomVar.toInt(), \
TYPE=v.altAllele.category(), \
CATO=if (isMissing(va.allele.annotation.predictions.CATO.score)) "\\\N" else str(va.allele.annotation.predictions.CATO.score), \
eigen=if (isMissing(va.allele.annotation.predictions.Eigen.EigenRaw)) "\\\N" else str(va.allele.annotation.predictions.Eigen.EigenRaw), \
polyPhen=if (isMissing(va.allele.annotation.vep.predictions.PolyPhen)) "\\\N" else str(va.allele.annotation.vep.predictions.PolyPhen), \
sift=if (isMissing(va.allele.annotation.vep.predictions.SIFT)) "\\\N" else str(va.allele.annotation.vep.predictions.SIFT), \
HGVSc=if (isMissing(va.allele.annotation.vep.HGVS.HGVSc)) "\\\N" else str(va.allele.annotation.vep.HGVS.HGVSc), \
HGVSp=if (isMissing(va.allele.annotation.vep.HGVS.HGVSp)) "\\\N" else str(va.allele.annotation.vep.HGVS.HGVSp), \
tgpAF=if (isMissing(va.allele.annotation.freqs.tgp.AF)) "\\\N" else str(va.allele.annotation.freqs.tgp.AF), \
hrcAF=if (isMissing(va.allele.annotation.freqs.hrc.AF)) "\\\N" else str(va.allele.annotation.freqs.hrc.AF), \
gnomadAF=if (isMissing(va.allele.annotation.freqs.gnomad.AF)) "\\\N" else str(va.allele.annotation.freqs.gnomad.AF), \
feature=orElse(va.allele.annotation.vep.consequences.feature.id, "\\\N"), \
consequences=orElse(va.allele.annotation.vep.consequences.consequences.mkString(","), "\\\N"), \
gene=orElse(va.allele.annotation.vep.cross_references.Ensembl_gene, "\\\N"), \
clinvar=orElse(va.allele.annotation.clinvar.ClinicalSignificance.mkString(","), "\\\N"), \
wasSplit=va.extra.wasSplit')

kt = kt.select([u'VARIANT', u'CHROMOSOME', u'START', u'REF', u'ALT', u'RSID', u'AC', u'AF', u'nHomRef', u'nHet', u'nHomVar', u'TYPE', 'CATO', 'eigen', 'sift', 'HGVSc', 'HGVSp', 'polyPhen', 'tgpAF', 'hrcAF', 'gnomadAF', 'feature', 'consequences', 'gene','clinvar', 'wasSplit'])


In [6]:
kt.export("/nvme/tmp/shusson/mgrb_sample.tsv")

2017-10-25 13:10:03 Hail: INFO: while writing:
    /nvme/tmp/shusson/mgrb_sample.tsv
  merge time: 367.510ms


In [ ]:
kt.show()

In [25]:
from pprint import pprint
pprint(vds.variant_schema)

Struct{
     variant: Struct{
         qual: Double,
         filters: Set[String],
         metrics: Struct{
             sample_counts: Struct{
                 female: Long,
                 male: Long,
                 total: Long
             },
             chromatid_counts: Struct{
                 female: Double,
                 male: Double,
                 total: Double
             },
             gatk: Struct{
                 AN: Int,
                 BaseQRankSum: Double,
                 ClippingRankSum: Double,
                 DB: Boolean,
                 DP: Int,
                 DS: Boolean,
                 END: Int,
                 ExcessHet: Double,
                 FS: Double,
                 HaplotypeScore: Double,
                 InbreedingCoeff: Double,
                 MQ: Double,
                 MQRankSum: Double,
                 NEGATIVE_TRAIN_SITE: Boolean,
                 POSITIVE_TRAIN_SITE: Boolean,
                 QD: Double,
                